# Explorando datos mundiales de COVID-19

In [ ]:
import numpy as np
import pandas as pd

### Preparación de datos

Importamos los datos a un dataframe de pandas

In [ ]:
df_confirmados = pd.read_csv('./data/covid19_confirmados.csv')
df_confirmados.head(5)

Crearemos un MultiIndex con las columnas 'Country/Region' y 'Province/State' para facilitar el proceso de agrupar las provincias de cada país en una sola fila.

In [ ]:
df_confirmados.set_index(keys=['Country/Region', 'Province/State'], drop=True,inplace=True)

In [ ]:
df_confirmados.head(5)

Eliminamos las columnas "Lat" y "Long":

In [ ]:
df_confirmados.drop(labels=['Lat', 'Long'], axis=1, inplace=True)

In [ ]:
df_confirmados.head(5)

Luego, reinterpretamos las etiquetas de fecha como Timestamps en lugar de simples objetos.

Para eso, le aplicamos a cada columna la función `pd.to_datetime`.

In [ ]:
df_confirmados.rename(mapper=pd.to_datetime, axis='columns');
df_confirmados.head(5)

Ahora, reducimos el índice Province/State, para que quede solo una fila por país:

In [ ]:
df_confirmados = df_confirmados.groupby(by='Country/Region').aggregate(np.sum)

In [ ]:
df_confirmados.head(5)